   # TRATAMIENTO Y ANÁLISIS DE BASE DE DATOS "USOS BICIMAD" vs Clima de Madrid

La base **'Bicis acumulados por día'** nos da la información desde junio de 2014 hasta diciembre de 2018. Nos brinda registros con información de fechas, *tipo de abonado*, y *cantidad de uso por abonado*. Inicialmente la idea es identificar como es el comportamiento de cada tipo de abonado por año, por mes y por día de semana. Antes de esto deberemos transformar la tabla y adaptarla a nuestras necesidades.

A continuación se cruza esta información con los datos del clima aprovechando que se analizan y manipulan en otro notebook.

## Librerías a importar

Realizamos las importaciones de las librerías que se van a utilizar. En este caso vamos a trabajar con:
    
    * Pandas
    * Pandasplot
    * Numpy
    * Seaborn
    * Matplotlib
    
 Estas librerías nos permitirán en su orden la manipulación de los datos y su posterior graficación.   

In [ ]:
from datetime import date, datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

## Extracción de los datos
El fichero *'bicis_usos_acumulado'* se encuentra enformato CSV por lo cual se se realizará la extracción con pandas read_csv.

In [ ]:
bicis_usos = pd.read_csv('../dat/bicis_usos_acumulado por dia.csv', sep=';', index_col=False)


## Inspección de datos

En este punto revisamos el tipo de datos que tenemos, el tamaño de la tabla, cantidad de columnas y filas.

Visualizamos las primeras 5 columnas y filas de la tabla. La columna index por defecto iniciará desde 'cero'.

In [ ]:
bicis_usos.head()

Visualizamos las últimas 5 columnas y filas de la tabla.

In [ ]:
bicis_usos.tail()

Revisamos los diferentes tipos de datos con los que contamos, ya advertimos que la columna DIA precisa conversión a formato
fecha. Los demás datos tal cual como se encuentran nos servirán.

In [ ]:
bicis_usos.dtypes

Revisamos demás dimensiones para obtener la cantidad de filas y columnas.

In [ ]:
bicis_usos.shape

Implementamos **.describe** para obtener la información relacionada con la distribución de los *datos, count, mean,std*.

In [ ]:
bicis_usos.describe()

## Transformación de los datos

La etapa de transformación de datos incluye:

* Cambio de etiquetas
* Transformación de tipo de datos
* Eliminación de columnas sin importancia
* Generación de nuevas columnas formuladas

Cambiamos los nombres de todas las columnas por nombres mas descriptivos y/o fáciles de manejar

In [ ]:
bicis_usos.columns=['fecha', 'abono_anual', 'abono_ocasional', 'total_abonos', 'a_a_acum', 'a_o_acum', 'total_acum', 'na']
bicis_usos.head()

### Cambio de formato de fecha

En este caso se aplicará una función apply para modificar la columna fecha la cual por defecto está en formato Object, y la necesitamos en 'datetime'.

In [ ]:
def cast_date(date):
    return datetime.strptime(date, '%d/%m/%Y')

bicis_usos['fecha'] = bicis_usos.fecha.apply(cast_date)

Ahora queda en formato **Datetime**.

In [ ]:
bicis_usos.dtypes

### Eliminación de columna

Usamos la función .Drop para eliminar las columnas que a nuestro modo de ver no aportan nada a nuestros intereses.

In [ ]:
bicis_usos.head(1)

In [ ]:
bicis_usos.drop(bicis_usos.columns[[4,5,6,7]], axis=1, inplace=True)

Confirmamos eliminación de columna.

In [ ]:
bicis_usos.head(1)

### Creación de nuevas columnas

A partir de la columna fecha vamos a extraer una columna con el 'año' y otra con el 'mes'. Adicionalmente vamos a generar una
función que nos cree una columna con el día de la semana. En ambos casos usaremos 'apply' con la función lambda.

Para mejor tratamiento de los datos generamos dos columnas nuevas:'ano' y 'mes' con lambda.

In [ ]:
bicis_usos['ano'] = bicis_usos.apply(lambda fila: fila.fecha.year, axis = 1)
bicis_usos['mes'] = bicis_usos.apply(lambda fila: fila.fecha.month, axis = 1)

Es necesario identificar los días de la semana, por lo que incorporamos la función 'get_day_of_week' en linea con apply.
En el mismo proceso creamos una nueva columna la cual llamaremos *'dia_semana'*.

In [ ]:
def get_day_of_week(row):    
    day = row.fecha.weekday()
    if day == 0:
        return 'Lunes'
    elif day == 1:
        return 'Martes'
    elif day == 2:
        return 'Miércoles'
    elif day == 3:
        return 'Jueves'
    elif day == 4:
        return 'Viernes'
    elif day == 5:
        return 'Sabádo'
    elif day == 6:
        return 'Domingo'
    
bicis_usos['dia_semana'] = bicis_usos.apply( get_day_of_week, axis = 1)
    
bicis_usos.head()

Con el fin de obtener un vista de los datos que nos permita generar otras opciones de gráficas y análisis,
aplicamos la función MELT

Reasignamos nombres nuevamente a las columnas de la nueva versión de la tabla.

In [ ]:
bicis_usos_melt = pd.melt(bicis_usos, id_vars=["ano", "mes", "dia_semana"], value_vars=['abono_anual', 'abono_ocasional'], )
bicis_usos_melt.columns = ['ano', 'mes', 'dia_semana', 'tipo_usuario', 'cantidad_usos']
bicis_usos_melt.head()

## Gráficas

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import rcParams

rcParams['figure.figsize'] = 14, 10
sns.set_context('talk') #Ampliación de gráficas

Los datos obtenidos cuentan con pocas variables categóricas, razón por la cual optamos inicialmente por mostrar dos gráficas que contienen el nivel de uso de los tipos de usuarios como lo es el *Usuario de abono anual* y el *Usuario de abono ocasional*, que además manejan frecuencias de usos muy distintas, por lo tanto es mejor trabajar cada tipo de usuario por separado para poder visualizar a mejor escala los comportamientos de cada uno.

Con esta gráfica tipo **lineplot** podemos ver por separado las diferencias de escala en los desempeños por año de cada tipo de usuario.

In [ ]:
out = sns.lineplot(data=bicis_usos_melt, x='ano', y='cantidad_usos', hue='tipo_usuario')

plt.xlabel('Años')
plt.ylabel('Cantidad Usos')
plt.title('Usos Abonado Anual Vs. Abonado Ocasional')
plt.show()

Otra vision con Catplot retratanto la diferencia de escalas de cada tipo de usuario pero en una sola gráfica.

In [ ]:
sns.set(style="ticks")
out = sns.catplot(x="ano", y="cantidad_usos", col="tipo_usuario", data=bicis_usos_melt)


****GRÁFICAS DE ESCALA ANUAL****

Con esta gráfica **Boxplot** por *Uso Total* observamos la relación de la variable de **'usos'** con la variable **'año'**. En otras palabras como el sistema aumenta anualmente su nivel de demanda. En junio de 2014 se inician actividades y rápidamente pasa de 7.000 usos aprox. a 15.000 usos en 2015. En 2016 se presenta una disminución y en los siguientes años retoma nuevamente la senda creciente. Podemos observar también que no se presentan datos átipicos importantes y además como en el 3er cuartil se van concentrando para los últimos 2 años una demanda de 8000 a 11000 usos por año.

* En 2016 se presenta la municipalización del sistema, que en su transición deja por fuera por concepto de mantenimiento mas de 1000 bicicletas. Además de una deserción de mas de 4.000 abonados, esto atribuible al incertidumbre generada por la operación de compra del ayuntamiento. fuente=  https://www.elmundo.es/madrid/2017/09/02/59a996ba22601d99558b463e.html

In [ ]:
out = sns.boxplot(x="ano", y="total_abonos" , data=bicis_usos)
plt.xlabel('Años')
plt.ylabel('Cantidad Usos')
plt.title('Uso Total Abonados Anuales')
plt.show()

El siguiente análisis lo aplicamos con una gráfica tipo **Lineplot** y en ella podemos observar de forma individualizada el comportamiento de los **Abonados Anuales** durante los 5 años: un arranque el primer año que en menos de 6 meses obtiene aprox 6.000 usos, aumenta a mas de 8.000 usos en 2015 y en 2016 cae para reponerse nuevamente a partir de 2017.

In [ ]:
out = sns.lineplot(data=bicis_usos_melt[(bicis_usos_melt.tipo_usuario=='abono_anual')] , x='ano',
                   y='cantidad_usos', hue='tipo_usuario', ci=0, err_style='bars',palette="Set1",
                   linewidth=2.5, legend='full')

plt.xlabel('Años')
plt.ylabel('Cantidad Usos')
plt.title('Cantidad Uso Abonado Anual por Año')
plt.show()

Ahora para completar el analisis de la relación de las variables de *Cantidad de uso* por *Año*, observamos a través de otro **lineplot**, pero en este caso aplicado a los **Abonados ocasionales**, que el comportamiento tiene una tendencia parecida a los **Abonados Anuales** al principio. Que aunque la escala es totalmente diferente, podemos ver el mismo ascenso el primer y segundo año, para caer nuevamente en 2016, pero a diferencia del otro tipo de usuario, este no mantiene un ascenso en 2017 y 2018, sino que se mantiene estable entre los 175 y 200 usos anuales. 
Revisando la baja demanda de estos abonados, se obtiene información que el sistema está enfocado en los abonados anuales, entendiendo que estos realemte hacen parte del sistema multimodal que quiere implementar el ayuntamiendo, dandole al sistema un caracter de transporte complementario y no potenciar p.e, el uso del sistema por el turismo.

In [ ]:
out = sns.lineplot(data=bicis_usos_melt[(bicis_usos_melt.tipo_usuario=='abono_ocasional')] , x='ano',
                   y='cantidad_usos', hue='tipo_usuario', ci=0, err_style='bars',palette="Set2",
                   linewidth=2.5, legend='full')

plt.xlabel('Años')
plt.ylabel('Cantidad Usos')
plt.title('Cantidad de Usos Abonado Ocasional por Año')
plt.show()

****GRAFICAS DE ESCALA DIARIA****

***MEDIA DE USO ABONADOS ANUALES POR DÍA DE LA SEMANA***
A continuación un ***Barplot***. 
Esta gráfica contiene la media de los usos por días de los 5 años que tiene de vigencia el sistema.
Vemos como la usabilidad los **Abonados Anuales** tiene un comportamiento intenso durante los días hábiles, especialmente entre martes y jueves. Este desempeño contrasta con el de los **Abonados Ocasionales** que veremos a continuación.

In [ ]:
sns.barplot(x='dia_semana', y='abono_anual', estimator=np.mean, data=bicis_usos, palette="Set3", ci=0)
plt.xlabel('Día de la Semana')
plt.ylabel('Cantidad Usos')
plt.title('Media de Uso de Abonados Anuales por Día de la Semana')
plt.show()

En la siguiente gráfica **Barplot** podemos observar el comportamiento de la media de uso por día de la semana de los **Abonados Ocasionales**. Claramente los fines de semana es cuando aumenta el uso por parte de este tipo de usuario, que puede ser asociado al turismo. Tener claro este comportamiento es fundamental, aunque las cifras puedan parecer muy bajas, sí que se detecta una tendencia que podría ser explotada mas adelante con ofertas adaptadas a estos días.

In [ ]:
sns.barplot(x='dia_semana', y='abono_ocasional', estimator= np.mean, data=bicis_usos,palette="Set3", ci=0)
plt.xlabel('Día de la Semana')
plt.ylabel('Cantidad Usos')
plt.title('Media de Uso de Abonados Ocasionales por Día de la Semana')
plt.show()

****GRAFICAS DE ESCALA MENSUAL****

Para continuar con el análisis del comportamiento por **Mes** de los usuarios con **Abono Anual** usaremos gráficas tipo **Pointplot** que son ideales para puntualizar cortes. En ese caso observamos como el segundo trimestre presenta siempre subas en su comportamiento, también en los meses de *Agosto* y *Septiembre* se presentan aumentos todos los años. Y para cerrar las bajas que se presentan son siempre para verano (Agosto principalmente) y los meses de *Diciembre*  a *Marzo*, fácilmente atribuíble a épocas de vacaciones y las bajas temperaturas del invierno, respectivamente.

In [ ]:
out = sns.pointplot(data=bicis_usos_melt[bicis_usos_melt.tipo_usuario=='abono_anual'],x='mes',
                   y='cantidad_usos', hue='ano', ci=0, err_style='bars',palette="tab10",
                   linewidth=2.5, legend='full')

plt.xlabel('Mes')
plt.ylabel('Cantidad Usos')
plt.title('Uso Mensual de Abonado Anual por Mes')
plt.show()

En el caso de los **Abonados Ocasionales** los meses de *mayo, junio, septiembre y octubre* son los momentos de mayor demanda del sistema. Igualmente con caídas muy pronunciadas desde *diciembre* hasta comienzos de *marzo*. 

In [ ]:
out = sns.pointplot(data=bicis_usos_melt[bicis_usos_melt.tipo_usuario=='abono_ocasional'],x='mes',
                   y='cantidad_usos', hue='ano', ci=0, err_style='bars',palette="tab10",
                   linewidth=2.5, legend='full')

plt.xlabel('Mes')
plt.ylabel('Cantidad Usos')
plt.title('Uso Mensual Abonado Ocasional por Mes')
plt.show()

****CRUCE DE TABLAS (Merge)****

Para darle mas profundidad y contexto a la información sobre el uso de las bicicletas hemos optado por realizar cruces con el clima, entonces podremos ver la relación de los días de lluvia y la temperatura con el uso del sistema

Importamos una tabla del clima la cual trabajamos en otro *Notebook*. (**Clima Madrid.ipynb**)

In [ ]:
clima= pd.read_csv('../dat/all_weather_data.csv', sep=',', index_col=False)
clima.head()

Esta tabla también amerita modificar el formato de la fecha, para lo cual aplicamos función *datetime.strptime*.

In [ ]:
def cast_date(date):
    return datetime.strptime(date, '%Y-%m-%d')

clima['date'] = clima.date.apply(cast_date)

Ahora implementamos la función *merge* para cruzar la tabla clima con la tabla usos. Tabla que llamaremos **climabicis**

In [ ]:
climabicis=bicis_usos.merge(clima, left_on='fecha', right_on='date')
climabicis.head()

Inspeccionanamos nuevamente la tabla resultado del *merge* y además verificamos el tipo de datos obtenido.

In [ ]:
climabicis.tail()

Además realizamos la eliminación de las columnas que no necesitamos, para lo que nuevamente usamos la función *Drop*.

In [ ]:
climabicis.drop(climabicis.columns[[7,8,9,10,12,14,15,16]], axis=1, inplace=True)
climabicis.tail()

In [ ]:
climabicis.dtypes

Ahora solo falta convertir las columnas tipo float como **'abono_anual'**, **'Abono_ocasional'** y **'Año'** entre otros a números enteros(int). Para esto aplicamos función **lambda** por cada columna.

In [ ]:
climabicis['ano']= climabicis.apply(lambda fila: int(fila['ano']), axis=1)
climabicis['abono_anual']= climabicis.apply(lambda fila: int(fila['abono_anual']), axis=1)
climabicis['abono_ocasional']= climabicis.apply(lambda fila: int(fila['abono_ocasional']), axis=1)
climabicis['total_abonos']= climabicis.apply(lambda fila: int(fila['total_abonos']), axis=1)
climabicis['mes']= climabicis.apply(lambda fila: int(fila['mes']), axis=1)

Confirmamos el cambio de tipo y ya quedó listo para trabajar con gráficas.

In [ ]:
climabicis.dtypes

En esta gráfica detectamos claramente que si cruzamos el uso total del sistema con la temperatura encontraremos una relación directa: a mayor temperatura, mayor uso de bicicletas. Sin embargo, cuando la media de la temperatura supera los 25 grados, ya afecta negativamente el uso del sistema. Es entre 20 y 25 la condición ideal de temperatura para el uso del sistema. Lo que habíamos confirmado antes con el uso por mes, donde en los meses de invierno reflejan una gran disminución de usos.  

In [ ]:
out = sns.regplot(x='temp_media', y='total_abonos', data=climabicis )
out = sns.scatterplot(x='temp_media', y='total_abonos', data=climabicis , hue='temp_media', palette='YlOrRd')

plt.xlabel('Temperatura')
plt.ylabel('Cantidad Usos')
plt.title('Cantidad Uso Total Vs. Temperatura')
plt.show()

Para revisar la relación entre la lluvia y el uso de bicicletas del sistema, podemos ver que que esta variable no afecta directamente el sistema. Los días que hubo 'cero' lluvias, evidentemente hay uso del sistema, pero los días que llovió no se ve tan afectado dicho uso. Realmente llueve poco en Madrid y cuando sucede, no es impedimento. 

In [ ]:
out = sns.scatterplot(x='precipitation', y='total_abonos', data=climabicis, palette="BuGn")

plt.xlabel('Nivel de Precipitación')
plt.ylabel('Cantidad Usos')
plt.title('Cantidad Usos Totales Vs. Nivel de Precipitación')
plt.show()